In [1]:
import numpy as np
np.random.seed(42)

In [2]:
import pandas as pd
df = pd.read_parquet("user_item_ratio_stacked.parquet")
df.columns = ["userID", "itemID", "rating"]

In [3]:
filtered_df = df.groupby('userID')['itemID'].count().reset_index().loc[lambda x: x['itemID'] > 2]


In [4]:
df= df.loc[df.userID.isin(filtered_df.userID)]

In [21]:
df.columns = ['user', 'item', 'rating']

In [23]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

# Load your DataFrame with columns ['user', 'item', 'rating']
# Modify the file paths accordingly
# Example:
# df = pd.read_csv('your_data_file.csv')

# Getting the number of users and movies
nb_users = int(df['user'].nunique())
nb_movies = int(df['item'].nunique())

# Create a user-item matrix (ratings matrix)
def convert(data):
    new_data = []
    for id_users in range(1, nb_users + 1):
        id_movies = data['item'][data['user'] == id_users]
        id_ratings = data['rating'][data['user'] == id_users]
        ratings = np.zeros(nb_movies)
        ratings[id_movies - 1] = id_ratings
        new_data.append(list(ratings))
    return new_data

# Convert the DataFrame to the user-item matrix
data = convert(df)

# Convert the data into Torch tensors
data = torch.FloatTensor(data)

# Creating the architecture of the Neural Network
class SAE(nn.Module):
    def __init__(self, ):
        super(SAE, self).__init__()
        self.fc1 = nn.Linear(nb_movies, 20)
        self.fc2 = nn.Linear(20, 10)
        self.fc3 = nn.Linear(10, 20)
        self.fc4 = nn.Linear(20, nb_movies)
        self.activation = nn.Sigmoid()

    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)
        return x

sae = SAE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr=0.01, weight_decay=0.5)

nb_epoch = 256
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0.
    for id_user in range(nb_users):
        input = Variable(data[id_user]).unsqueeze(0)
        target = input.clone()
        if torch.sum(target.data > 0) > 0:
            output = sae(input)
            target.requires_grad = False
            output[target == 0] = 0
            loss = criterion(output, target)
            mean_corrector = nb_movies / float(torch.sum(target.data > 0) + 1e-10)
            loss.backward()
            train_loss += np.sqrt(loss.data * mean_corrector)
            s += 1.
            optimizer.step()
    print('epoch: ' + str(epoch) + ' loss: ' + str(train_loss / s))

test_loss = 0
s = 0.
for id_user in range(nb_users):
    input = Variable(data[id_user]).unsqueeze(0)
    target = input.clone()
    if torch.sum(target.data > 0) > 0:
        output = sae(input)
        target.requires_grad = False
        output[target == 0] = 0
        loss = criterion(output, target)
        mean_corrector = nb_movies / float(torch.sum(target.data > 0) + 1e-10)
        test_loss += np.sqrt(loss.data * mean_corrector)
        s += 1.
print('test loss: ' + str(test_loss / s))


epoch: 1 loss: tensor(0.3192)
epoch: 2 loss: tensor(0.2394)
epoch: 3 loss: tensor(0.2369)
epoch: 4 loss: tensor(0.2351)
epoch: 5 loss: tensor(0.2339)
epoch: 6 loss: tensor(0.2328)
epoch: 7 loss: tensor(0.2317)
epoch: 8 loss: tensor(0.2310)
epoch: 9 loss: tensor(0.2299)
epoch: 10 loss: tensor(0.2291)
epoch: 11 loss: tensor(0.2281)
epoch: 12 loss: tensor(0.2273)
epoch: 13 loss: tensor(0.2263)
epoch: 14 loss: tensor(0.2254)
epoch: 15 loss: tensor(0.2244)
epoch: 16 loss: tensor(0.2235)
epoch: 17 loss: tensor(0.2225)
epoch: 18 loss: tensor(0.2216)
epoch: 19 loss: tensor(0.2206)
epoch: 20 loss: tensor(0.2196)
epoch: 21 loss: tensor(0.2188)
epoch: 22 loss: tensor(0.2178)
epoch: 23 loss: tensor(0.2171)
epoch: 24 loss: tensor(0.2161)
epoch: 25 loss: tensor(0.2154)
epoch: 26 loss: tensor(0.2144)
epoch: 27 loss: tensor(0.2136)
epoch: 28 loss: tensor(0.2126)
epoch: 29 loss: tensor(0.2128)
epoch: 30 loss: tensor(0.2115)
epoch: 31 loss: tensor(0.2109)
epoch: 32 loss: tensor(0.2096)
epoch: 33 loss: t